In [1]:
!pip install --upgrade --quiet openai duckpy

In [6]:
import openai
import os
from openai import OpenAI
with open('gram_token.txt', 'r') as file:
    os.environ['OPENAI_API_KEY'] = file.read().strip()
    
openai.api_key = os.getenv('OPENAI_API_KEY')

print(openai.__version__)

1.40.6


In [7]:
import json 
from duckpy import Client
import ast

client = openai.OpenAI()
duckduckgo_client = Client()

def duck_search(query) -> str:
    """ Run a duckduckgo search"""
    output = duckduckgo_client.search(query)
    return str(output)

In [11]:
completion = client.chat.completions.create(
    model = "gpt-4",
    temperature = 0,
    functions=[
         {
                "name": "duck_search",
                "description": "Used to search online",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Translate the Korean content into English input query",
                        },
                    },
                    "required": ["query"],
                },
            }
    ],
     messages=[
            {"role": "system", "content": "You must use the `duck_search` function to get information."},
            {"role": "user", "content": "조지 클루니 생일이 언제야?"},
        ]
)

print(completion)

ChatCompletion(id='chatcmpl-9w52sO3VJpFq83D5NIZczRhxYE7CD', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "query": "George Clooney birthday"\n}', name='duck_search'), tool_calls=None))], created=1723629446, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=81, total_tokens=100))


In [14]:
#function call success

message = completion.choices[0].message

if message.function_call :
    function_name = message.function_call.name
    args = ast.literal_eval(message.function_call.arguments)
    function_response = duck_search(
        query = args.get("query")
    )
    print(function_response)
    completion_final = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": "조지 클루니 생일이 언제야?"},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )
    print(completion_final)
    print(completion_final.choices[0].message.content)

[]
ChatCompletion(id='chatcmpl-9w55E4CbuaV81oijfcm9edzbuimqW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='조지 클루니의 생일은 1961년 5월 6일입니다.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723629592, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_3aa7262c27', usage=CompletionUsage(completion_tokens=21, prompt_tokens=86, total_tokens=107))
조지 클루니의 생일은 1961년 5월 6일입니다.


In [13]:
example_user_input = "런던 날씨가 안 좋다면서?"

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": example_user_input}],
    functions=[
      {
          "name": "get_different_responses",
          "description": "답변을 여러가지 말투로 하기",
          "parameters": {
              "type": "object",
              "properties": {
                  "response_up": {
                      "type": "string",
                      "description": "답변을 높임말로 바꿈",
                  },
                  "response_down": {
                      "type": "string",
                      "description": "답변을 반말로 바꿈",
                  }
              },
              "required": ["response_up", "response_down"],
          },
      }
    ],
    function_call={"name": "get_different_responses"},
)

reply_content = completion.choices[0].message
print(reply_content)
print(reply_content.function_call.arguments)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "response_up": "네, 맞습니다. 런던의 날씨가 좋지 않다는 정보가 있습니다.",\n  "response_down": "그래, 런던 날씨가 좋지 않다고 들었어."\n}', name='get_different_responses'), tool_calls=None)
{
  "response_up": "네, 맞습니다. 런던의 날씨가 좋지 않다는 정보가 있습니다.",
  "response_down": "그래, 런던 날씨가 좋지 않다고 들었어."
}
